In [17]:
from sklearn.cluster import KMeans
import cv2
import os
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches

import json
import cv2

import sys
from PIL import Image
sys.setrecursionlimit(10000)


In [18]:
def fetch_img(imgNo, base_path='im_val/'):
    img_name = nameD[imgNo][0]['file_name']
    img_path = base_path + img_name
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    segs = [annD[imgNo][i]['segmentation'][0] for i in range(len(annD[imgNo]))]
    xs, ys = [], []
    for seg in segs:
        x, y = [seg[i] for i in range(0, len(seg), 2)], [seg[i] for i in range(1, len(seg), 2)]
        xs.append(x)
        ys.append(y)
    return (img, (xs, ys))


def draw_ann(img, coords):
    xs, ys = coords
    imgs = [img, img]
    n_row = 1
    n_col = len(imgs)
    _, axs = plt.subplots(n_row, n_col, figsize=(12, 12))
    axs = axs.flatten()
    cnt = 0
    for img, ax in zip(imgs, axs):
        ax.imshow(img)
        if cnt % 2:
            for i in range(len(xs)):
                x, y = xs[i], ys[i]
                
                plt.fill(x, y)
        cnt += 1
    plt.show()

def make_bboxes(resimg):
    res = np.array(resimg)
    visited = [[False for _ in range(len(res))] for __ in range(len(res[0]))]
    nums_init = [0, 0, len(res), len(res)] # xmx, ymx, xmn, ymn
    step = 2
    bboxes = []
    n = len(res)
    nums = nums_init.copy()
    for i in range(n):
        for j in range(n):
            if res[i][j][0] != 0:
                if not visited[i][j]:
                    dfs(res, i, j, nums, visited, step)
                    bboxes.append(nums)
                    nums = nums_init.copy()
    return bboxes

def plot_bboxes(resimg, bboxes):
    fig, ax = plt.subplots()
    ax.imshow(resimg)
    cols = list('bgrcmyk')
    cnt = 0
    for bbox in bboxes:
        imx, jmx, imn, jmn = bbox
        # plt.plot(jmx, imx, jmn, imn, marker='o', color=cols[cnt % len(cols)])
        rect = patches.Rectangle((jmn, imn), (jmx-jmn), (imx-imn), linewidth=1, edgecolor=cols[cnt % len(cols)], facecolor='none')
        ax.add_patch(rect)
        cnt += 1
    plt.show()

In [19]:
fpath = r'C:\Users\hiran\Desktop\MM5565_PROJECT\Dataset\512x\iii'

try:
  os.chdir(fpath)
except FileNotFoundError:
  pass

with open('label/val.json') as f:
  imgD = json.load(f)

annD = dict()
for i in imgD['annotations']:
    imgN = i['image_id']
    if imgN in annD:
        annD[imgN].append(i)
    else:
        annD[imgN] = [i]

nameD = dict()
for i in imgD['images']:
    imgN = i['id']
    if imgN in nameD:
        nameD[imgN].append(i)
    else:
        nameD[imgN] = [i]

imgidD=dict()
for i in nameD :
   imgidD[nameD[i][0]["file_name"]]=i

C:\Users\hiran\Desktop\MM5565_PROJECT\Dataset\512x\i


In [21]:


def transform(filename,res,fr) :
    img_id=imgidD[filename]
    img,coords=fetch_img(img_id)
    ret=[]
    for i in range(len(annD[img_id])):
        trans_img=[[0 for _ in range(res)] for l in range(res)]
        bbox=annD[img_id][i]['bbox']
        if bbox[2]*bbox[3]> res*res*fr :
            for i in range(4):
                bbox[i]=int(bbox[i]//1)
            for x in range(bbox[0],bbox[0]+bbox[2]):
                for y in range(bbox[1],bbox[1]+bbox[3]) :
                    trans_img[y][x]=img[y][x][0]
    
            ret.append(np.array(trans_img))
            
    return np.array(ret)

In [22]:

res=512
sav_dir=r"C:\Users\hiran\Desktop\MM5565_PROJECT\Dataset\masked_dataset\iii\im_val"
os.chdir(fpath)
inum=1

for filnam in os.listdir(os.path.join(fpath,"im_val")):
    
    for im in transform(filnam,res,.05):
        imgname="img"+str(inum)+".png"
        os.chdir(sav_dir)
        cv2.imwrite(imgname, im)
        os.chdir(fpath)
        inum=inum+1
 
    